In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
pd.options.display.max_columns = 100
from google.cloud import bigquery
from datetime import datetime
from datetime import date, timedelta

import sys
sys.path.append('/home/jupyter/aaaie-customer-personas/src')

from customer_personas.bq_utils import export_dataframe_to_bq, upload_data_to_gcs

In [2]:
client = bigquery.Client(location="northamerica-northeast1")

## cto-wln-sa-data-pr-bb5283

In [3]:
query = \
'''
SELECT *
FROM `cto-wln-sa-data-pr-bb5283`.INFORMATION_SCHEMA.SCHEMATA
'''

In [4]:
table_df = client.query(query).to_dataframe()

In [5]:
table_df.schema_name.unique()

array(['ukraine_calls', 'hs_churn', 'customer_personas_temp',
       'mobility_prospecting', 'ext_table', 'measurements_data',
       '5G_speed_tiers', 'demostats_2021', 'crime_stat_model',
       'vertex_ai_pipelines', 'ccai_insights_for_churn',
       'wln_churn_hsia_model', 'featurehub_postal_demogr', 'app_cat_map',
       'campaigns_emails', 'proactive_assurance', 'solution_sell',
       'int_res_cust_cust',
       'export_evaluated_data_items_untitled_1662742715070_2022_09_09T10_02_48_809Z',
       'wln_churn_account', 'wifi_plus', 'eval_prospecting',
       'ffh_multicultural', 'ffh_multicultural_pipeline',
       '4th_carrier_analysis', 'internet_upsell', 'featurehub_ban_demogr',
       'wln_churn_account_analysis', 'feature_store_ban_clickstream',
       'customer_personas_reports', 'roaming_pre_trip_data',
       'copper_fiber_migration', 'tmp', 'log_table',
       'nearmap_ai_insights', 'crime_stats_bi_weekly_campaigns',
       'stage_age_model', 'devops', 'tos', 'content_cre

In [ ]:
table_column_query = \
'''
SELECT
  *
FROM
  `{project_id}.{dataset_name}`.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS
'''

In [ ]:
project_column_info_df = pd.DataFrame()
for index, row in table_df.iterrows():
    query_ = table_column_query.format(project_id = row['catalog_name'],
                                       dataset_name = row['schema_name'])
    df = client.query(query_).to_dataframe()
    project_column_info_df = project_column_info_df.append(df)

In [ ]:
project_column_info_df.reset_index(drop=True)

## Datahub Tables

In [6]:
datahub_dataset_names = pd.read_csv('datahub_table_names.csv')

In [7]:
datahub_dataset_names.head(2)

,datahub_table_names
0,ent_chat
1,ent_cntct_evnt


In [8]:
datahub_column_query = \
'''
SELECT
  *
FROM
  `cio-datahub-enterprise-pr-183a.{dataset_name}`.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS
'''

In [9]:
datahub_column_info_df = pd.DataFrame()
for index, row in datahub_dataset_names.iterrows():
    query_ = datahub_column_query.format(dataset_name = row['datahub_table_names'])
    try:
        df = client.query(query_).to_dataframe()
        datahub_column_info_df = datahub_column_info_df.append(df)
    except: 
        continue

In [10]:
datahub_column_info_df.reset_index(drop=True)

,table_catalog,table_schema,table_name,column_name,field_path,data_type,description,collation_name,rounding_mode
0,cio-datahub-enterprise-pr-183a,ent_chat,bq_agent_chat_intractn,agnt_chat_intractn_id,agnt_chat_intractn_id,STRING,Unique interaction id assigned for chat intera...,NULL,None
1,cio-datahub-enterprise-pr-183a,ent_chat,bq_agent_chat_intractn,agnt_chat_intractn_start_ts,agnt_chat_intractn_start_ts,DATETIME,Start time of the chat interaction,NULL,None
2,cio-datahub-enterprise-pr-183a,ent_chat,bq_agent_chat_intractn,agnt_chat_intractn_mod_ts,agnt_chat_intractn_mod_ts,DATETIME,Time when the chat interaction was modified,NULL,None
3,cio-datahub-enterprise-pr-183a,ent_chat,bq_agent_chat_intractn,agnt_chat_intractn_end_ts,agnt_chat_intractn_end_ts,DATETIME,End time of the chat interaction,NULL,None
4,cio-datahub-enterprise-pr-183a,ent_chat,bq_agent_chat_intractn,agnt_chat_intractn_stat,agnt_chat_intractn_stat,INT64,Status assigned to the chat interaction,NULL,None
...,...,...,...,...,...,...,...,...,...
81226,cio-datahub-enterprise-pr-183a,src_tmods,bq_position_extract,personnel_sub_area_id,personnel_sub_area_id,NUMERIC,None,NULL,None
81227,cio-datahub-enterprise-pr-183a,src_tmods,bq_position_extract,personnel_sub_area_cd,personnel_sub_area_cd,STRING,None,NULL,None
81228,cio-datahub-enterprise-pr-183a,src_tmods,bq_position_extract,personnel_sub_area_txt,personnel_sub_area_txt,STRING,None,NULL,None
81229,cio-datahub-enterprise-pr-183a,src_tmods,bq_position_extract,location_cd,location_cd,STRING,None,NULL,None


In [13]:
BQ_TABLE_ID= 'cto-wln-sa-data-pr-bb5283.ref_table.datahub_schema_info'
export_dataframe_to_bq(datahub_column_info_df.reset_index(drop=True),
                       table_id=BQ_TABLE_ID,
                       schema_list=[],
                       generate_schema=True)

[SchemaField('table_catalog', 'STRING', 'NULLABLE', None, (), None), SchemaField('table_schema', 'STRING', 'NULLABLE', None, (), None), SchemaField('table_name', 'STRING', 'NULLABLE', None, (), None), SchemaField('column_name', 'STRING', 'NULLABLE', None, (), None), SchemaField('field_path', 'STRING', 'NULLABLE', None, (), None), SchemaField('data_type', 'STRING', 'NULLABLE', None, (), None), SchemaField('description', 'STRING', 'NULLABLE', None, (), None), SchemaField('collation_name', 'STRING', 'NULLABLE', None, (), None), SchemaField('rounding_mode', 'STRING', 'NULLABLE', None, (), None)]
Loaded 81231 rows and 9 columns to cto-wln-sa-data-pr-bb5283.ref_table.datahub_schema_info
